<a href="https://colab.research.google.com/github/Rahul-Ag/Learners-Space-Machine-Learning/blob/main/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import deeplake
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score, average_precision_score

In [ ]:
# Load dataset from DeepLake
ds = deeplake.load("hub://activeloop/ham10000")

Opening dataset in read-only mode as you don't have write permissions.


\

In [ ]:
# Preprocess image function
def preprocess_image(image, img_size=(224, 224)):
    image = cv2.resize(image, img_size)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.0
    return image

In [ ]:
# Load and preprocess the data
img_size = (224, 224)
def generate_images_labels(ds, img_size=(224, 224)):
    images = []
    labels = []
    for i in range(int(len(ds['images'])/6)):
        image = ds['images'][i].numpy()
        label = ds['lesion_categories'][i].numpy()  # Use the 'lesion_categories' tensor for labels
        image = preprocess_image(image, img_size)
        images.append(image)
        labels.append(label)
    return np.array(images), np.array(labels)

images, labels = generate_images_labels(ds, img_size)

In [ ]:
# Convert labels to categorical format
num_classes = len(np.unique(labels))
labels = tf.keras.utils.to_categorical(labels, num_classes=num_classes)

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Create ImageDataGenerators
train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(train_images, train_labels, batch_size=32, shuffle=True)
validation_generator = val_datagen.flow(val_images, val_labels, batch_size=32, shuffle=False)

In [ ]:
# Load the EfficientNetB0 model with pre-trained weights
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [ ]:
# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=15
)

In [ ]:
# Evaluate the model on validation data
val_predictions = model.predict(val_images)

In [ ]:
# Calculate metrics
accuracy = accuracy_score(np.argmax(val_labels, axis=1), np.argmax(val_predictions, axis=1))
auc = roc_auc_score(val_labels, val_predictions, multi_class='ovo')
f1 = f1_score(np.argmax(val_labels, axis=1), np.argmax(val_predictions, axis=1), average='weighted')
mean_ap = average_precision_score(val_labels, val_predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'AUC: {auc}')
print(f'F1 Score: {f1}')
print(f'Mean Average Precision: {mean_ap}')

In [ ]:
# Save the model
model.save('medical_image_classification_model.h5')